# CIFAR-10

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import sys

sys.path.append('../')
sys.path.append('../src/')

import src.general as general
import src.dataset_models as data
import src.metrics as metrics
import src.evaluation as eval
import src.plot as plot
import src.compression.distillation as distill
import src.compression.pruning as prune


/home/abel/anaconda3/envs/pytorch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010])
])


In [4]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)


Files already downloaded and verified
Files already downloaded and verified


In [5]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)


In [6]:
dataset = data.DataSet(
        name="CIFAR-10",
        criterion=F.cross_entropy,
        metric=metrics.accuracy,
        train_loader=trainloader,
        test_loader=testloader,
)

In [13]:
dataset = data.supported_datasets["CIFAR-10"]

In [8]:
import models.resnet as rs


resnet = rs.resnet18()

# Replace the last fully connected layer with a new one
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, 10)


In [9]:
# Load the fine-tuned weights
resnet.load_state_dict(torch.load('../models/resnet18.pt'))

<All keys matched successfully>

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)


Evaluate before compressing

In [15]:
before_results = eval.get_results(resnet, dataset)

plot.print_results(**before_results)

Test: 100%|██████████| 157/157 [00:18<00:00,  8.29it/s]


Test loss: -2.0095
Test score: 38.7440
==================================== RESULTS ===================================
Loss: -2.009520
Score: 38.744029
Time per data point: 7.5364 ms
Model Size: 42.69 MB
Number of parameters: 11173962
Number of FLOPs: 140786688
Number of MACs: 140848128


# Compression

# Pruning

Structured pruning example.

In [ ]:
# Prunin

## Distillation

In [ ]:
settings = {
    # "performance_target": 99,
    "fineTune": False,
    "epochs": 3,
}

distilled_model = distill.perform_distillation(resnet, dataset, settings)

Settings: {'fineTune': False, 'epochs': 3}
============================= CREATING STUDENT MODEL ===========================
Fine-tuning: False


============================ PERFORMING DISTILLATION ===========================
Epoch: 0


Distillation Training: 100%|██████████| 391/391 [03:42<00:00,  1.76it/s]


Distillation loss: 4.014499505435544e+32


Distillation Validation: 100%|██████████| 79/79 [00:06<00:00, 11.50it/s]


Test loss: 4.700567996448561e+16, Test score: 9.958465189873417
Epoch: 1


...

Distillation loss: 3.965401642908699e+32


Distillation Validation: 100%|██████████| 79/79 [00:08<00:00,  9.83it/s]


Test loss: 4.700568007321895e+16, Test score: 9.958465189873417
Epoch: 2


Distillation Training:   9%|▊         | 34/391 [00:22<03:56,  1.51it/s]


KeyboardInterrupt: 

In [ ]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  